In [ ]:
# Import
import MDAnalysis as mda
from MDAnalysis import analysis
from MDAnalysis.analysis import contacts
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from colour import Color
import matplotlib.pyplot as plt
directory = r'C:\Users\emfla\Desktop\Favour\CodeStuff'

In [ ]:
u = mda.Universe(r'C:\Users\emfla\Desktop\Favour\CodeStuff\step7_1.part0010.gro',r'C:\Users\emfla\Desktop\Favour\CodeStuff\shorter.xtc')

In [ ]:
proteins = u.select_atoms('resname NIC')
membrane = u.select_atoms('resname SAPS SAPE CHL1 POPC OSM SAPC SOPE SLPC SAPI')

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import distances
import numpy as np
import pandas as pd

# Load your universe
u = mda.Universe(r'C:\Users\emfla\Desktop\Favour\CodeStuff\step7_1.part0010.gro',r'C:\Users\emfla\Desktop\Favour\CodeStuff\shorter.xtc')

# Select protein and lipid atoms
proteins = u.select_atoms('resname NIC')
lipids = u.select_atoms('resname SAPS SAPE CHL1 POPC OSM SAPC SOPE SLPC SAPI')

# Initialize a dictionary to store the summed contact counts for each protein atom
contact_sums = {atom_id: 0 for atom_id in proteins.ids}

# Function to calculate contacts
def contact_calc(protein=proteins, membrane=lipids, cont_dist=6, start=0, stop=-1, skip=1):
    print('Atoms to analyse: ', len(proteins.atoms))
    print('With: ', len(membrane.atoms), ' targets')
    
    # Iterate over all frames in the trajectory
    for ts in u.trajectory[start:stop:skip]:
        print(f'Analyzing frame at time {u.trajectory.time}')
        
        # Calculate contacts for each protein atom
        for X in proteins.ids:
            atom = u.select_atoms(f'id {X}')
            dist_array = distances.distance_array(atom.positions, membrane.positions)
            contact_count = np.count_nonzero(dist_array <= cont_dist)
            contact_sums[X] += contact_count  # Accumulate contact counts

# Run the contact calculation
contact_calc(proteins, lipids)

# Convert the results to a pandas DataFrame
results_df = pd.DataFrame(list(contact_sums.items()), columns=['Atom_ID', 'Total_Contacts'])

# Save the results to a CSV file (optional)
results_df.to_csv('protein_lipid_contacts.csv', index=False)

# Display the results
print(results_df)
# This gives the total contacts for each individual protein atom with ALL lipid atoms

In [ ]:
# This cell only exists to inform the information needed in the cell below #
for name in np.unique(lipids.resnames):
    print('For the lipid species ' + str(name))
    # Get the first residue ID for the current lipid species
    resid = str(u.select_atoms('resname ' + name).resids[0])
    # Select atoms for the specific residue and get the number of atoms
    num_atoms = len(u.select_atoms('resname ' + name + ' and resid ' + resid).atoms)
    print('The number of atoms in that species is: ', num_atoms)
    print('-----------------------------')

In [ ]:
# Load your universe
u = mda.Universe(r'C:\Users\emfla\Desktop\Favour\CodeStuff\step7_1.part0010.gro',r'C:\Users\emfla\Desktop\Favour\CodeStuff\shorter.xtc')

# Define lipid species and their atom counts
# This information we can take from the cell above if you have a different membrane
lipid_species = {
    'SAPS': {'selection': 'resname SAPS', 'num_atoms': 133},
    'SAPE': {'selection': 'resname SAPE', 'num_atoms': 131},
    'CHL1': {'selection': 'resname CHL1', 'num_atoms': 74},  # Example, adjust as needed
    'POPC': {'selection': 'resname POPC', 'num_atoms': 134},  # Example, adjust as needed
    'OSM': {'selection': 'resname OSM', 'num_atoms': 131},   # Example, adjust as needed
    'SAPC': {'selection': 'resname SAPC', 'num_atoms': 140},
    'SOPE': {'selection': 'resname SOPE', 'num_atoms': 131},
    'SLPC': {'selection': 'resname SLPC', 'num_atoms': 138},  # Example, adjust as needed
    'SAPI': {'selection': 'resname SAPI', 'num_atoms': 143},
}

# Initialize a dictionary to store the summed contact counts for each protein atom and lipid atom type
contact_sums = {atom_id: {} for atom_id in proteins.ids}

# This will generate a contact for each individual protein atom with all lipid atoms!
# It was taking me around 11 seconds per frame!
def contact_calc(protein=proteins, cont_dist=6, start=0, stop=-1, skip=1):
    print('Atoms to analyse: ', len(proteins.atoms))
    
    # Record the start time
    start_time = time.time()
    print(f"Calculation started at: {print_current_time()}")
    
    # Precompute a mapping of atom IDs to local indices for each lipid species
    lipid_mappings = {}
    atom_names = {}  # Store atom names for each species
    for species, data in lipid_species.items():
        selection = data['selection']
        lipids = u.select_atoms(selection)
        lipid_mappings[species] = {atom.id: idx for idx, atom in enumerate(lipids)}
        
        # Store atom names for the species
        residue = lipids.residues[0]  # Use the first residue to get atom names
        atom_names[species] = [atom.name for atom in residue.atoms]
    
    # Iterate over all frames in the trajectory
    for ts in u.trajectory[start:stop:skip]:
        frame_start_time = time.time()
        print(f'Analyzing frame at time {u.trajectory.time}')
        
        # Calculate contacts for each protein atom and lipid species
        for X in proteins.ids:
            atom = u.select_atoms(f'id {X}')
            for species, data in lipid_species.items():
                selection = data['selection']
                num_atoms = data['num_atoms']
                lipids = u.select_atoms(selection)
                
                # Initialize atom type counts if not already done
                for atom_type in range(num_atoms):
                    atom_name = atom_names[species][atom_type]  # Get the actual atom name
                    column_name = f'{species}_{atom_name}'
                    if column_name not in contact_sums[X]:
                        contact_sums[X][column_name] = 0
                
                # Calculate distances
                dist_array = distances.distance_array(atom.positions, lipids.positions)
                
                # Iterate over each lipid residue and atom type
                for residue in lipids.residues:
                    residue_atoms = residue.atoms
                    for atom_type in range(num_atoms):
                        lipid_atom = residue_atoms[atom_type]  # Atom indices start at 0
                        
                        # Get the local index from the precomputed mapping
                        lipid_atom_idx = lipid_mappings[species][lipid_atom.id]
                        
                        # Check if the distance is within the contact distance
                        if dist_array[0, lipid_atom_idx] <= cont_dist:
                            atom_name = atom_names[species][atom_type]  # Get the actual atom name
                            column_name = f'{species}_{atom_name}'
                            contact_sums[X][column_name] += 1
        
        # Calculate time taken for the current frame
        frame_time = time.time() - frame_start_time
        print(f"Time taken for frame {u.trajectory.frame}: {frame_time:.2f} seconds")
    
    # Record the end time
    end_time = time.time()
    print(f"Calculation ended at: {print_current_time()}")
    
    # Calculate total time elapsed
    total_time = end_time - start_time
    print(f"Total time elapsed: {total_time:.2f} seconds")
    
    # Calculate time per frame
    num_frames = len(u.trajectory[start:stop:skip])
    time_per_frame = total_time / num_frames
    print(f"Time per frame: {time_per_frame:.2f} seconds")

# Run the contact calculation
contact_calc(proteins)

# Convert the results to a pandas DataFrame
results_list = []
for atom_id, lipid_contacts in contact_sums.items():
    row = {'Atom_ID': atom_id}
    row.update(lipid_contacts)
    results_list.append(row)

results_df = pd.DataFrame(results_list)

# Save the results to a CSV file (optional)
results_df.to_csv('protein_lipid_atom_type_contacts.csv', index=False)

# Display the results
print(results_df)

In [ ]:
# This will sum over the appropriate number of lipids for each species
# E.g. if we have 8 POPC, it will add up each corresponding atom so there is just a single lipid's worth of columns
# We still have all protein atoms as each row

def split_dataframe_by_species(input_csv, output_prefix='lipid_species_'):
    """
    Splits a DataFrame stored in a CSV file into separate CSV files for each lipid species.

    Parameters:
        input_csv (str): Path to the input CSV file.
        output_prefix (str): Prefix for the output CSV files. Default is 'lipid_species_'.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Get the list of all columns in the DataFrame
    columns = df.columns
    
    # Extract unique lipid species from the column names
    lipid_species = set(col.split('_')[0] for col in columns if '_' in col)
    
    # Iterate over each lipid species
    for species in lipid_species:
        # Select columns for the current species
        species_columns = [col for col in columns if col.startswith(species + '_')]
        
        # Create a new DataFrame with the selected columns
        species_df = df[['Atom_ID'] + species_columns]
        
        # Save the DataFrame to a CSV file
        output_csv = f"{output_prefix}{species}.csv"
        species_df.to_csv(output_csv, index=False)
        print(f"Saved {output_csv} with {len(species_columns)} columns.")

# Example usage
split_dataframe_by_species('protein_lipid_atom_type_contacts.csv') # at the moment this also outsputs a _Atom.csv, you can ignore that

In [ ]:
# This will sum up so we can only account for a single protein instead
# The output will be a .csv file with a number of rows equal to the number of atoms of a single protein

def sum_atoms_across_proteins(input_csv, output_csv, num_proteins=10, atoms_per_protein=26):
    """
    Sums the values for each atom across all proteins and saves the result to a new CSV file.

    Parameters:
        input_csv (str): Path to the input CSV file.
        output_csv (str): Path to the output CSV file.
        num_proteins (int): Number of proteins. Default is 10.
        atoms_per_protein (int): Number of atoms per protein. Default is 26.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Initialize a dictionary to store the summed values for each atom
    summed_data = {col: [] for col in df.columns}
    
    # Loop over each atom (0 to 25)
    for atom_idx in range(atoms_per_protein):
        # Select rows for the current atom across all proteins
        rows = range(atom_idx, len(df), atoms_per_protein)
        atom_rows = df.iloc[rows]
        
        # Sum the rows for the current atom across all proteins
        summed_row = atom_rows.sum(axis=0)
        
        # Append the summed row to the dictionary
        for col in df.columns:
            summed_data[col].append(summed_row[col])
    
    # Create a new DataFrame from the summed data
    summed_df = pd.DataFrame(summed_data)
    
    # Save the summed DataFrame to a CSV file
    summed_df.to_csv(output_csv, index=False)
    print(f"Saved summed atom values to {output_csv}")

# Example usage
sum_atoms_across_proteins('protein_lipid_atom_type_contacts.csv', 'summed_atom_contacts.csv')

In [ ]:
# This will create an individual .csv file for each lipid species
# Because it uses the previous cell, it leaves us

def split_summed_dataframe_by_species(input_csv, output_prefix='lipid_species_'):
    """
    Splits a summed DataFrame into separate CSV files for each lipid species.

    Parameters:
        input_csv (str): Path to the input CSV file.
        output_prefix (str): Prefix for the output CSV files. Default is 'lipid_species_'.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Get the list of all columns in the DataFrame
    columns = df.columns
    
    # Extract unique lipid species from the column names
    lipid_species = set(col.split('_')[0] for col in columns if '_' in col)
    
    # Iterate over each lipid species
    for species in lipid_species:
        # Select columns for the current species
        species_columns = [col for col in columns if col.startswith(species + '_')]
        
        # Create a new DataFrame with the selected columns
        species_df = df[['Atom_ID'] + species_columns]
        
        # Save the DataFrame to a CSV file
        output_csv = f"{output_prefix}{species}.csv"
        species_df.to_csv(output_csv, index=False)
        print(f"Saved {output_csv} with {len(species_columns)} columns.")

# Example usage
split_summed_dataframe_by_species('summed_atom_contacts.csv')

In [ ]:
pip install colour

In [ ]:
# Function to sum contacts across lipid atoms and generate histograms with RGB colors
# This will output a .txt file that has the colour index based on the total number of contacts that protein atom has with ALL lipids of that species
# This is the .txt file we can load into PYMOL directly to colour the atoms of the protein correctly
def process_lipid_csv(input_csv, output_csv, color_file, bins=50):
    """
    Processes a lipid-specific CSV file to sum contacts, generate histograms, and map to RGB colors.

    Parameters:
        input_csv (str): Path to the input CSV file (e.g., 'SAPI.csv').
        output_csv (str): Path to the output CSV file with summed contacts.
        color_file (str): Path to the output text file with RGB colors.
        bins (int): Number of bins for the histogram. Default is 30.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Sum contacts across all lipid atoms for each protein atom
    df['Total_Contacts'] = df.iloc[:, 1:].sum(axis=1)
    
    # Save the summed contacts to a new CSV file
    summed_df = df[['Atom_ID', 'Total_Contacts']]
    summed_df.to_csv(output_csv, index=False)
    print(f"Saved summed contacts to {output_csv}")
    
    # Generate a histogram for the summed contacts
    contact_values = summed_df['Total_Contacts'].values
    hist, bin_edges = np.histogram(contact_values, bins=bins, range=(0, np.max(contact_values)))
    
    # Create a color gradient from blue to red
    colors = list(Color('Blue').range_to(Color('Red'), bins))
    
    # Map each protein atom to an RGB color based on its histogram bin
    color_index = []
    for value in contact_values:
        bin_index = np.digitize(value, bin_edges) - 1
        if bin_index >= bins:  # Handle values at the upper edge
            bin_index = bins - 1
        color_index.append(colors[bin_index].rgb)
    
    # Save the RGB colors to a text file
    with open(color_file, 'w') as f:
        for color in color_index:
            f.write(f"[{color}],\n")
    print(f"Saved RGB colors to {color_file}")

    # And now to make it exactly as pymol likes it        
    # Open the file for reading
    with open(color_file, 'r') as f:
        # Read the contents of the file
        contents = f.read()

    # Replace all occurrences of '(' with '['
    new_contents = contents.replace("(","[")
    new_contents = new_contents.replace(")","]")
    # Open the file for writing
    with open('PYMOL_Prot_'+color_file, "w+") as f:
        # Write the modified contents back to the file
        f.write(new_contents)
    # And close the file when we are done with it
    f.close()
    
    # Plot the histogram (optional)
    plt.hist(contact_values, bins=bins, range=(0, np.max(contact_values)), edgecolor='black')
    plt.title('Histogram of Summed Contacts')
    plt.xlabel('Summed Contacts')
    plt.ylabel('Frequency')
    plt.show()

# Example usage
lipid_species = ['SAPS', 'SAPE', 'CHL1', 'POPC', 'OSM', 'SAPC', 'SOPE', 'SLPC', 'SAPI']
for species in lipid_species:
    input_csv = f'lipid_species_{species}.csv'
    output_csv = f'summed_contacts_{species}.csv'
    color_file = f'color_index_{species}.txt'
    process_lipid_csv(input_csv, output_csv, color_file)

In [ ]:
# Function to sum contacts across protein atoms and generate histograms with RGB colors
# This is the same as above except it will generate a colour index for the lipid species instead
# I am pretty sure this is legit and we don't have to redo the contact calculations with proteins and lipids swapped around...
def process_lipid_csv_for_coloring(input_csv, output_csv, color_file, bins=30):
    """
    Processes a lipid-specific CSV file to sum contacts, generate histograms, and map to RGB colors.

    Parameters:
        input_csv (str): Path to the input CSV file (e.g., 'SAPI.csv').
        output_csv (str): Path to the output CSV file with summed contacts.
        color_file (str): Path to the output text file with RGB colors.
        bins (int): Number of bins for the histogram. Default is 30.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)
    
    # Sum contacts across all protein atoms for each lipid atom
    total_contacts = df.iloc[:, 1:].sum(axis=0)
    
    # Create a DataFrame for the summed contacts
    summed_df = pd.DataFrame({
        'Lipid_Atom': df.columns[1:],  # Lipid atom names (e.g., SAPS_C3, SAPS_C4, ...)
        'Total_Contacts': total_contacts
    })
    
    # Save the summed contacts to a new CSV file
    summed_df.to_csv(output_csv, index=False)
    print(f"Saved summed contacts to {output_csv}")
    
    # Generate a histogram for the summed contacts
    contact_values = summed_df['Total_Contacts'].values
    hist, bin_edges = np.histogram(contact_values, bins=bins, range=(0, np.max(contact_values)))
    
    # Create a color gradient from blue to red
    colors = list(Color('Blue').range_to(Color('Red'), bins))
    
    # Map each lipid atom to an RGB color based on its histogram bin
    color_index = []
    for value in contact_values:
        bin_index = np.digitize(value, bin_edges) - 1
        if bin_index >= bins:  # Handle values at the upper edge
            bin_index = bins - 1
        color_index.append(colors[bin_index].rgb)
    
    # Save the RGB colors to a text file
    with open(color_file, 'w') as f:
        for color in color_index:
            f.write(f"[{color}],\n")
    print(f"Saved RGB colors to {color_file}")

    # And now to make it exactly as pymol likes it        
    # Open the file for reading
    with open(color_file, 'r') as f:
        # Read the contents of the file
        contents = f.read()

    # Replace all occurrences of '(' with '['
    new_contents = contents.replace("(","[")
    new_contents = new_contents.replace(")","]")
    # Open the file for writing
    with open('PYMOL_'+color_file, "w+") as f:
        # Write the modified contents back to the file
        f.write(new_contents)
    # And close the file when we are done with it
    f.close()
    
    # Plot the histogram (optional)
    plt.hist(contact_values, bins=bins, range=(0, np.max(contact_values)), edgecolor='black')
    plt.title('Histogram of Summed Contacts')
    plt.xlabel('Summed Contacts')
    plt.ylabel('Frequency')
    plt.show()

# Example usage
lipid_species = ['SAPS', 'SAPE', 'CHL1', 'POPC', 'OSM', 'SAPC', 'SOPE', 'SLPC', 'SAPI']
for species in lipid_species:
    input_csv = f'lipid_species_{species}.csv'
    output_csv = f'summed_contacts_lipid_{species}.csv'
    color_file = f'color_index_lipid_{species}.txt'
    process_lipid_csv_for_coloring(input_csv, output_csv, color_file)

In [ ]:
# Useful information for PYMOL

# Get unique residue names
unique_residue_names = set(res.resname for res in proteins.residues) | set(res.resname for res in lipids.residues)

# Loop over each unique residue name and print details
for residue_name in unique_residue_names:
    residue_group = u.select_atoms(f"resname {residue_name}").residues
    residue_count = len(residue_group)
    atoms_per_residue = len(residue_group[0].atoms) if residue_count > 0 else "N/A"
    first_atom_index = residue_group[0].atoms[0].index if residue_count > 0 and len(residue_group[0].atoms) > 0 else "N/A"

    print(f"Residue Name: {residue_name} | Residue Count: {residue_count} | "
          f"Atoms Per Residue: {atoms_per_residue} | Index of Atom 1 Residue 1: {first_atom_index}")

In [ ]:
# To then colour stuff in PYMOL:
# OPEN UP PYMOL, LOAD IN THE RELEVANT .GRO/.PDB FILE FOR YOUR SYSTEM
# AND THEN DO
# run pymol_colours.py
# IN THE PYMOL COMMAND LINE. YOU NEED TO TWEAK THE VALUES IN THE .PY SCRIPT TO BE RELEVANT FOR THE RESIDUE YOU ARE INTERESTED IN
# first_atom_index, color_file
# ARE THE TWO THINGS THAT NEED CHANGING
# LET ME KNOW IF IT DOESN'T WORK. TRY THIS FIRST WITH A SHORTER TRAJECTORY!